<a href="https://www.kaggle.com/code/aneeshgrover/binary-classification-tss24-competition-3?scriptVersionId=185680988" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Thapar Summer School 2024 Competition 3: Binary Classification

## Objective:
### Prediction of the probability whether a person is a smoker based on given features in the dataset. 
### Evaluations are on the basis of the ROC curve (AUC-ROC).

## Importing Libraries

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Exploration and Visualization 

In [ ]:
train = pd.read_csv("/kaggle/input/tss24-competition-3/train.csv")
test = pd.read_csv("/kaggle/input/tss24-competition-3/test.csv")

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train.dtypes

In [ ]:
train.isna().sum()

In [ ]:
plt.figure(figsize = (8, 6))
sns.countplot(x = 'smoking', data = train)

plt.title('Distribution of Smoking')
plt.xlabel('Smoking')
plt.ylabel('Count')
plt.show()

## Preprocessing

In [ ]:
X = train.drop(['id', 'smoking'], axis = 1)
Y = train['smoking']

In [ ]:
numericalColumnsX = X.select_dtypes(include = ['float64', 'int64']).columns
numericalColumnTest = test.select_dtypes(include = ['float64', 'int64']).columns

Scaler = StandardScaler()
X[numericalColumnsX] = Scaler.fit_transform(X[numericalColumnsX])
test = test.drop('id', axis = 1)
test = Scaler.transform(test)

## Defining and training our model

In [ ]:
model_lgbm = LGBMClassifier(random_state = 42)
model_lgbm.fit(X, Y)
predictions_lgbm = model_lgbm.predict_proba(test)[:, 1]

model_xgb = XGBClassifier(random_state=42, use_label_encoder=False)
model_xgb.fit(X, Y)
predictions_xgb = model_xgb.predict_proba(test)[:, 1]

ensemble_predictions = (predictions_lgbm + predictions_xgb) / 2

In [ ]:
smth = pd.read_csv("/kaggle/input/tss24-competition-3/test.csv")

Submission = pd.DataFrame({
    'id' : smth['id'], 
    'smoking' : ensemble_predictions
})

Submission.to_csv('Submission.csv', index = False)
print("Submission file created successfully.")


In [ ]:
print("a")